<H2>MADATORY PYTHON LIBRARIES</H2>

In [1]:
import pandas as pd
import ftputil
import numpy as np
import os
import datetime 

/home/protllan/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Python 2 suport will be removed in ftputil 4.0.0
  from ipykernel import kernelapp as app


<h2>TIME UPDATE SELECTION CRITERIA</h2>

If you had a look to the notebook dedicated to the In Situ netCDFs selection criteria (here), selecting by asociated time update is available for files in all directories:

In [2]:
data = { 'Selection criteria': ['time update'], 'index_history.txt': [' ✔'], 'index_monthly.txt': [' ✔'],'index_latest.txt': [' ✔ ✔']}
pd.DataFrame(data=data)

,Selection criteria,index_history.txt,index_latest.txt,index_monthly.txt
0,time update,✔,✔ ✔,✔


<H2>SET OUTPUT DIRECTORY</H2>

In [3]:
output_directory = os.getcwd() #default to current working directory

<h2> SET YOUR CREDENTIALS</h2>

In [4]:
user = '' #type CMEMS user name
password = '' #type CMEMS password

<h2> TARGET A PRODUCT, HOST AND INDEX FILE</h2>

In [5]:
product_name = 'INSITU_MED_NRT_OBSERVATIONS_013_035' #type aimed In Situ product 
host = 'nrt.cmems-du.eu' #type aimed host (nrt.cmems-du.eu or my.cmems-du)
index_file = 'index_latest.txt' #type aimed index file 

<h2> SPECIFY DATE TRESHOLD </h2>

In [6]:
date_format = "%Y-%m-%dT%H:%M:%SZ" 
date_treshold= datetime.datetime.strptime('2018-05-01T00:00:00Z', date_format)

<h2> DOWNLOAD THE FILES UPDATED AFTER THE ABOVE DATE TRESHOLD</h2>

In [8]:
#connect to CMEMS FTP
with ftputil.FTPHost(host, user, password) as ftp_host: 
    
    #open the index file to read
    with ftp_host.open("Core"+'/'+product_name+'/'+index_file, "r") as indexfile:
        
        #read the index file as a comma-separate-value file
        index = np.genfromtxt(indexfile, skip_header=6, unpack=False, delimiter=',', dtype=None, names=['catalog_id', 'file_name','geospatial_lat_min', 'geospatial_lat_max', 'geospatial_lon_min','geospatial_lon_max','time_coverage_start', 'time_coverage_end', 'provider', 'date_update', 'data_mode', 'parameters'])
        
        #loop over the lines/netCDFs and download the most sutable ones for you
        for netCDF in index:
            
            #getting ftplink, filepath and filename
            ftplink = netCDF['file_name'].decode('utf-8')
            filepath = '/'.join(ftplink.split('/')[3:len(ftplink.split('/'))])
            ncdf_file_name = ftplink[ftplink.rfind('/')+1:]
            
            #download netCDF if meeting selection criteria
            time_update = datetime.datetime.strptime(netCDF['date_update'].decode('utf-8'), date_format)
            if time_update > date_treshold: 
                if ftp_host.path.isfile(filepath):
                    cwd = os.getcwd()
                    os.chdir(output_directory)
                    ftp_host.download(filepath, ncdf_file_name)  # remote, local
                    os.chdir(cwd)

/home/protllan/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: DeprecationWarning: `use_list_a_option` will default to `False` in ftputil 4.x.x
  from ipykernel import kernelapp as app
